In [2]:
ls

 驱动器 F 中的卷是 存储
 卷的序列号是 C14D-581B

 F:\Codes\Jupyter\190919_DataMiningHW\project3 的目录

2019/11/15  19:01    <DIR>          .
2019/11/15  19:01    <DIR>          ..
2019/11/14  15:25    <DIR>          .ipynb_checkpoints
2019/11/14  20:18    <DIR>          __pycache__
2019/11/12  20:00            11,672 analyze.md
2019/11/15  19:01            38,575 db_data_compute.ipynb
2019/11/08  09:09             4,038 insert_mysql.ipynb
2019/11/13  09:59            56,146 main.ipynb
2019/11/14  20:20            56,341 main_v1_3649.ipynb
2019/11/14  22:44            44,220 main_v2.ipynb
2019/11/14  19:19             6,653 my_model.py
2019/11/14  20:20    <DIR>          storage
               7 个文件        217,645 字节
               5 个目录 406,666,260,480 可用字节


## 0.1 超参数

In [3]:
import numpy as np
import pandas as pd
import pymysql
import torch
from tqdm import tqdm

BATCH_SIZE = 8192
DB_NAME = 'zhihu2019_dataset'
DO_DATASET_BALANCE = True
EPOCH = 32
LR = 0.01
TRAIN_ALL = True

PKL_DIR_READ = './storage/train_param.pkl'
PKL_DIR_OUT = './storage/train_param.pkl'
RESULT_DIR_OUT = './storage/result.csv'

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

## 0.2 数据库函数

In [4]:
class DB():
    def __init__(self, host='localhost', port=3306, db=DB_NAME, user='root', passwd='123456', charset='utf8'):
        self.conn = pymysql.connect(host=host, port=port, db=db, user=user, passwd=passwd, charset=charset)   
        self.cur = self.conn.cursor(cursor = pymysql.cursors.DictCursor)

    def __enter__(self): 
        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.commit()
        self.cur.close()
        self.conn.close()

## 1.1 加载数据集

取训练集。

In [5]:
with DB() as db:
    db.execute('select * from train')
    train = pd.DataFrame(db.fetchall())

InterfaceError: (0, '')

取测试集。

In [39]:
with DB() as db:
    db.execute('select * from val')
    test = pd.DataFrame(db.fetchall())

合并二集。

In [40]:
col_list = ['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a', 'multi_feat_b',
             'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
train = train[col_list]
test = test[col_list]

In [41]:
data = pd.concat([train, test], axis=0)
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b',
       'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a',
       'multi_feat_b', 'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt',
       'u_answer', 'u_good_answer', 'u_recommand_answer', 'u_image_answer',
       'u_video_answer', 'u_word_avg', 'u_like_avg', 'u_unlike_avg',
       'u_comment_avg', 'u_collect_avg', 'u_thanks_avg', 'u_report_avg',
       'u_nohelp_avg', 'u_oppose_avg', 'q_answer', 'q_good_answer',
       'q_recommand_answer', 'q_image_answer', 'q_video_answer', 'q_word_avg',
       'q_like', 'q_comment', 'q_collect', 'q_thanks'],
      dtype='object')

## 1.2 处理数据集

离散特征的`LabelEncoder`编码。

In [42]:
from sklearn.preprocessing import LabelEncoder

# 离散值编码
class_feat = [#'用户id','问题id',
              'sex', 'access_freq','multi_feat_a','multi_feat_b',
              'multi_feat_c','multi_feat_d','multi_feat_e']
encoder = LabelEncoder()
for i in class_feat:
    data[i] = encoder.fit_transform(data[i])
    
data.iloc[:10]

,q_id,u_id,do_answer,invite_hour,follow_topic_hit,interest_topic_hit,sex,access_freq,bin_feat_a,bin_feat_b,...,q_answer,q_good_answer,q_recommand_answer,q_image_answer,q_video_answer,q_word_avg,q_like,q_comment,q_collect,q_thanks
0,Q1493039281,M64135255,None,-0.741275,-0.438202,1.318110,2,4,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Q2023398782,M2536956560,None,1.558090,-0.438202,-0.282984,2,0,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Q4151338694,M3294926344,None,0.319970,-0.438202,-0.282984,0,4,0.820988,-0.672968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Q3271436624,M3744310794,None,-1.625650,-0.438202,-0.282984,1,3,-1.218040,-0.672968,...,4.0,0.0,0.0,0.0,0.0,51.0,1.0,5.0,1.0,3.0
4,Q3314287018,M1349051752,None,1.027470,-0.438202,-0.282984,0,4,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Q4214103875,M2007129506,None,-0.033778,-0.438202,-0.282984,1,0,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Q1421177878,M3927950819,None,0.143096,-0.438202,-0.282984,0,0,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Q3598252818,M2871943120,None,-0.741275,-0.438202,-0.282984,1,4,-1.218040,1.485950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Q568518135,M998566127,None,0.850593,1.308280,-0.282984,2,2,0.820988,-0.672968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Q2242868437,M1307039867,None,-0.387527,-0.438202,-0.282984,0,0,0.820988,-0.672968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.3 包装数据集

In [43]:
import numpy as np

if DO_DATASET_BALANCE:
    temp = data[data['do_answer'] == 1]  # 用于均衡的数据
    print('用于均衡的数据有 %d 条.' % len(temp))
    train_x = data[data['do_answer']>-1].append([temp, temp, temp])
else:
    train_x = data[data['do_answer']>-1]
train_x, train_y = train_x[col_list[3:]].values, train_x[['do_answer']].values[:, 0]

test_x = data[data['do_answer']>-1]
test_x, test_y = test_x[col_list[3:]].values, test_x[['do_answer']].values[:, 0]
print('ok.')

用于均衡的数据有 0 条.
ok.


In [44]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, mode='train'):
        self.mode = mode
        if mode == 'train':
            self.data = {
                'x': train_x,
                'y': train_y,
            }
        elif mode == 'test':
            self.data = {
                'x': test_x,
                'y': test_y,
            }
        else:
            raise
            
    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]
        
    def __len__(self):
        return len(self.data['x'])
    
    
print('ok.')

ok.


In [46]:
from torch.utils.data import random_split

full_train_dataset = MyDataset(mode='train')
test_dataset = MyDataset(mode='test')
train_size, val_size, test_size = 0, 0, 0

if TRAIN_ALL:
    data_loader = {
        'train': DataLoader(dataset=full_train_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=5,
                            shuffle=True),
        'test': DataLoader(dataset=test_dataset,
                           batch_size=BATCH_SIZE,
                           num_workers=5,
                           shuffle=False),
    }
    train_size = len(full_train_dataset)
    test_size = len(test_dataset)
else:
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    data_loader = {
        'train': DataLoader(dataset=train_dataset, 
                            batch_size=BATCH_SIZE, 
                           num_workers=5,
                            shuffle=True),
        'val': DataLoader(dataset=val_dataset, 
                          batch_size=BATCH_SIZE,
                           num_workers=5,
                          shuffle=False),
        'test': DataLoader(dataset=test_dataset,
                           batch_size=BATCH_SIZE,
                           num_workers=5,
                           shuffle=False),
    }
    train_size = len(train_dataset)
    val_size = len(val_dataset)
    test_size = len(test_dataset)
    
print('train_size:', len(train_dataset))
print('val_size:', len(val_dataset))
print('test_size:', len(test_dataset))

ValueError: num_samples should be a positive integer value, but got num_samples=0

## 2.1 构建网络

In [48]:
from my_model import MyNet_v2

net = MyNet_v2(40).to(DEVICE)
net

MyNet_v2(
  (embed_sex): Embedding(3, 2)
  (embed_multi_a): Embedding(2561, 13)
  (embed_multi_b): Embedding(291, 6)
  (embed_multi_c): Embedding(428, 7)
  (embed_multi_d): Embedding(1556, 11)
  (bn0): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn7): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

## 2.2 优化器、损失函数

In [49]:
from torch import optim, nn

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)

'ok'

'ok'

## 2.3 训练

In [25]:
import math

# 计算一轮enumerate的长度
train_len = math.ceil(train_size / BATCH_SIZE)
val_len = math.ceil(val_size / BATCH_SIZE)
test_len = math.ceil(test_size / BATCH_SIZE)

print('train_len:', train_len)
print('val_len:', val_len)
print('test_len:', test_len)

train_len: 927
val_len: 232
test_len: 140


In [68]:
from tqdm import tqdm

for epoch in range(EPOCH):
    count, running_correct, running_loss = 0, 0, 0
    
    net.train()
    with tqdm(total=train_len) as pbar:
        for step, (bx, by) in enumerate(data_loader['train']):
            # 训练
            bx = bx.float().to(DEVICE)
            by = by.long().to(DEVICE)

            prediction = net(bx)
            loss = loss_function(prediction, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 统计成效
            pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
            my_loss = float(loss.data.cpu().numpy())
            my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
            my_acc = my_correct / BATCH_SIZE
            count += 1
            running_loss += my_loss
            running_correct += my_correct
            
            # 更新进度条
            pbar.update(1)
            pbar.set_description('Epoch: %d, Loss: %.4f, Acc: %.4f' % (
                                  epoch, 
                                  my_loss, 
                                  my_acc))
    
    # 输出一轮结果
    print('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
           epoch,
           running_loss / count,
           running_correct / train_size,
           optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 保存参数
    torch.save(net.state_dict(), PKL_DIR_OUT)
    
    # lr scheduler
    scheduler.step()

Epoch: 0, Loss: 0.6504, Acc: 0.0389: 100%|██████████| 1585/1585 [06:53<00:00,  3.83it/s]


Epoch: 0, Loss: 0.6321, Acc: 0.6247, Lr: 0.00010


Epoch: 1, Loss: 0.6156, Acc: 0.0436: 100%|██████████| 1585/1585 [08:07<00:00,  3.25it/s]


Epoch: 1, Loss: 0.6315, Acc: 0.6251, Lr: 0.00010


Epoch: 2, Loss: 0.6852, Acc: 0.0387: 100%|██████████| 1585/1585 [07:31<00:00,  3.51it/s]


...

Epoch: 12, Loss: 0.6302, Acc: 0.6264, Lr: 0.00010


Epoch: 13, Loss: 0.6463, Acc: 0.0405: 100%|██████████| 1585/1585 [06:03<00:00,  4.37it/s]


Epoch: 13, Loss: 0.6301, Acc: 0.6266, Lr: 0.00010


Epoch: 14, Loss: 0.6314, Acc: 0.6270:  44%|████▍     | 698/1585 [03:00<03:48,  3.88it/s]


KeyboardInterrupt: 

## 2.4 存储网络

In [69]:
torch.save(net.state_dict(), PKL_DIR_OUT)

## 4.1 导出 test 结果

In [10]:
result = data[np.isnan(data['是否回答'])]
result.iloc[-1]

问题id          Q2590286630
用户id          M3168697168
邀请创建时间           D3873-H9
性别                      1
是否回答                  NaN
用户二分类特征a           0.7121
用户二分类特征b        -0.652324
用户二分类特征c        -0.201699
用户二分类特征d        -0.591583
用户二分类特征e        -0.283857
用户多分类特征a             1092
用户多分类特征b              115
用户多分类特征c                6
用户多分类特征d              980
用户多分类特征e         0.343009
盐值                1.73947
访问评率            -0.995415
邀请问题间隔-day      -0.173304
关注话题命中数         -0.438016
Name: 1141682, dtype: object

In [26]:
result['是否回答']

0          0
1          0
2          0
3          0
4          0
          ..
1141678    0
1141679    0
1141680    0
1141681    0
1141682    0
Name: 是否回答, Length: 1141683, dtype: int64

In [27]:
from tqdm import tqdm
from my_model import MyNet

net = MyNet().to(DEVICE)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))
net.eval()

res_list = []
with tqdm(total=test_len) as pbar:
    for step, (bx, by) in enumerate(data_loader['test']):
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)
        
        prediction = net(bx)
        
        # 计算结果
        pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
        res_list.extend(pre.cpu().numpy())
        pbar.update(1)
        
result.loc[:, '是否回答'] = res_list
result[['是否回答']] = result[['是否回答']].astype(int)  # 数据类型转换为int

'ok'

100%|██████████| 140/140 [00:34<00:00,  4.02it/s]


In [29]:
result[['问题id', '用户id', '邀请创建时间', 
        '是否回答']].to_csv(RESULT_DIR_OUT, index=False, header=False, sep='\t')  # 导出数据

'ok'

'ok'